## Segmenting and Clustering Neighborhoods in Toronto

In [35]:
#Importing required dependencies
import pandas as pd 
import numpy as np
import requests
import io

Obtaining the data from the provided Wikipedia page and creating a pandas dataframe which excludes the cells in which a Borough  was not assigned

In [36]:
data_table=pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=862527922',header=0)
df1=data_table[0]
df_clean=df1[df1['Borough']!='Not assigned']
df_clean.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Combining into one row neighborhoods which belong to the same post code. These neighborhoods will be separated by a comma

In [37]:
df_join = df_clean.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [38]:
df_join.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Any neighborhood with the 'Not Assigned' id, will be replaced by the Borough name

In [39]:
df_join = df_final
df_final.loc[df_final['Neighbourhood']=='Not assigned','Neighbourhood']=df_final.loc[df_final['Neighbourhood']=='Not assigned']['Borough']

#An example of this case corrsponds to Postcode M7A:
df_final[df_final['Postcode']=='M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


Using the shape command to print the number of rows in the dataframe

In [40]:
df_final.shape[0]

103

Using the link to a csv file containing the geographical coordinates of each neighborhood, we can get the latitude and longitude

In [43]:
url="http://cocl.us/Geospatial_data"
answer=requests.get(url).content
geo=pd.read_csv(io.StringIO(answer.decode('utf-8')))
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We need to rename the first column of the acquired data frame in order to merge it with our existing data frame (df_final)

In [44]:
geo.columns = ['Postcode', 'Latitude', 'Longitude']
df_final = pd.merge(geo, df_final, on='Postcode')

Reordering the column names in the merged data frame, containing the latitude and longitude of each neighborhood

In [47]:
df_final = df_final[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
